In [16]:
import torch.nn as nn
import torchvision
import torch
from models import ParkingEfficientNet

In [17]:
# Dataset loading
folder_train = torchvision.datasets.ImageFolder(
    root="../train_images",
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        torchvision.transforms.ToTensor(),
    ])
)

train_loader = torch.utils.data.DataLoader(
    folder_train,
    batch_size=32,
    shuffle=True
)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ParkingEfficientNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)
model.train()

ParkingEfficientNet(
  (base): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (sc

In [19]:
# Train
num_episodes = 10

best_loss = float("inf")
for episode in range(num_episodes):
    episode_loss = 0

    for images, labels in train_loader:
        # Preprocess
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)

        # Reset gradients
        optimizer.zero_grad()

        # Feed stuff into model
        outputs = model(images)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss
        episode_loss += loss.item()

    print(f"Episode {episode}, loss: {episode_loss}")

    if episode_loss < best_loss:
        best_loss = episode_loss
        torch.save(model.state_dict(), "../models/parking_efficientnet.pth")
        print("\tSaving!")


Episode 0, loss: 25.844135630875826
	Saving!
Episode 1, loss: 1.6336776982061565
	Saving!
Episode 2, loss: 1.4684690876165405
	Saving!
Episode 3, loss: 0.6745982778375037
	Saving!
Episode 4, loss: 0.35480869276216254
	Saving!
Episode 5, loss: 0.3742260745202657
Episode 6, loss: 0.17789954008185305
	Saving!
Episode 7, loss: 0.09200585581129417
	Saving!
Episode 8, loss: 0.2002078363802866
Episode 9, loss: 1.2402177944823052
